In [1]:
import example_operations as eo

import sys
sys.path
sys.path.append('/home/masonmcgough/Workspace/HistoToolkit/app/tools')
import opnet

## Define functions

## Define network architecture

In [2]:
mynet1 = opnet.OpNet()

node1 = mynet1.add_node(eo.op_add, {'arg1': 4, 'arg2': 3}, ['add1'])
node1_split = mynet1.add_node(eo.op_split, {'arg1': None, 'n_outs': 2}, ['add1_1', 'add1_2'])
node2 = mynet1.add_node(eo.op_negate, {'arg1': None, 'negate': True}, ['negate1'])
node3 = mynet1.add_node(eo.op_add, {'arg1': None, 'arg2': None}, ['add2'])

conduit1 = mynet1.bind(node1, 'add1', node1_split, 'arg1')
conduit2 = mynet1.bind(node1_split, 'add1_1', node3, 'arg1')
conduit3 = mynet1.bind(node1_split, 'add1_2', node2, 'arg1')
conduit4 = mynet1.bind(node2, 'negate1', node3, 'arg2')


## Manually execute network 

In [3]:
print(node1.execute())
print(node1_split.execute())
print(node2.execute())
print(node3.execute())

print(conduit1.value)
print(conduit2.value)
print(conduit3.value)
print(conduit4.value)
print("# nodes: {0}".format(len(mynet1.nodes)))
print("# conduits: {0}".format(len(mynet1.conduits)))

{'add1': 7}
{'add1_1': 7, 'add1_2': 7}
{'negate1': 0}
{'add2': 7}
7
7
7
0
# nodes: 4
# conduits: 4


## Recurse from beginning through conduits, storing max depth at each node

In [4]:
# define root node(s) as any nodes with at least one non-Conduit input (or no inputs)
rootnodes = []
for node in mynet1.nodes:
    if any(not isinstance(i._value, opnet.Conduit) for i in node.params):
        rootnodes.append(node)
    
# walk through conduits (param -> outputs) and store depth (max path length) at each node
def walk_conduits(node, depth=0):
    new_depth = depth
    for output in node.outputs:
        if isinstance(output._value, opnet.Conduit):
            new_depth = min(depth, walk_conduits(output._value.output.node, depth+1))
        
    node.depth = new_depth
    return new_depth

for rootnode in rootnodes:
    walk_conduits(rootnode)
    
for node in mynet1.nodes:
    print(node)

Node:                     
	op: <function op_add at 0x7ff407596b90>                     
	params: [<opnet.Param instance at 0x7ff4074c62d8>, <opnet.Param instance at 0x7ff4074c6680>]                     
	outputs: [<opnet.Output instance at 0x7ff4074c66c8>]                     
	depth: 0
Node:                     
	op: <function op_split at 0x7ff407596cf8>                     
	params: [<opnet.Param instance at 0x7ff4074c6758>, <opnet.Param instance at 0x7ff4074c67a0>]                     
	outputs: [<opnet.Output instance at 0x7ff4074c67e8>, <opnet.Output instance at 0x7ff4074c6830>]                     
	depth: 0
Node:                     
	op: <function op_negate at 0x7ff407596d70>                     
	params: [<opnet.Param instance at 0x7ff4074c68c0>, <opnet.Param instance at 0x7ff4074c6908>]                     
	outputs: [<opnet.Output instance at 0x7ff4074c6950>]                     
	depth: 0
Node:                     
	op: <function op_add at 0x7ff407596b90>                  